In [1]:
import pickle

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from tensorflow import keras
import kerasncp as kncp

import os
from typing import Iterable, Dict
import tensorflow as tf
import kerasncp as kncp
from kerasncp.tf import LTCCell, WiredCfcCell
from tensorflow import keras
import numpy as np
from matplotlib.image import imread
from tqdm import tqdm
from PIL import Image
import pandas as pd
import time
from keras_models import generate_ncp_model
from train_test_loader import get_dataset_multi, get_val_dataset_multi

2024-09-17 15:02:08.843804: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
!export TF_CPP_MIN_LOG_LEVEL=2

In [3]:
def tlen(dataset):
    for (ix, _) in enumerate(dataset):
        pass
    return ix

In [4]:
training_root = "../fly_to_target_dataset/dataset"
val_root = "../fly_to_target_dataset/test_data"
DROPOUT = 0.1

DEFAULT_NCP_SEED = 22222

IMAGE_SHAPE = (144, 256, 3)
IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

batch_size = None
seq_len = 64
augmentation_params = None
single_step = False
no_norm_layer = False

decay_rate: float = 0.95
lr: float = 0.0001
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_steps=500,
                                                            # decay_rate=decay_rate, staircase=True)
#Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=lr)

In [5]:
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    mymodel = generate_ncp_model(seq_len, IMAGE_SHAPE, augmentation_params, batch_size, DEFAULT_NCP_SEED, single_step, no_norm_layer)
    mymodel.compile(optimizer=optimizer, loss="mean_squared_error", metrics=['mse'])
    mymodel.load_weights('model-ncp-val.hdf5')

    mymodel.summary()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2024-09-17 15:02:38.225039: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-09-17 15:02:38.226503: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-09-17 15:02:38.253044: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-17 15:02:38.253084: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: iras-hub
2024-09-17 15:02:38.253097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: iras-hub
2024-09-17 15:02:38.253261: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.54.14
2024-09-17 15:02:38.253517: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.54.14
2024-09-17 15:02:38.253538: I tensorflow/stream_executor/cu

OSError: Unable to open file (unable to open file: name = 'model-ncp-val.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
shift: int = 1
stride: int = 1
decay_rate: float = 0.95
val_split: float = 0.2
label_scale: float = 1
seq_len = 64
val_split: float = 0.1
label_scale: float = 1



In [ ]:
with tf.device('/cpu:0'):
    training_dataset = get_dataset_multi(training_root, IMAGE_SHAPE, seq_len, shift, stride, val_split, label_scale, extra_data_root=None)
    val_data = get_val_dataset_multi(val_root, IMAGE_SHAPE, seq_len, shift, stride, val_split, label_scale, extra_data_root=None)

print('\n\nTraining Dataset Size: %d\n\n' % tlen(training_dataset))

In [ ]:
print('load dataset shape', training_dataset.element_spec)
training_dataset = training_dataset.batch(64)
val_dataset = val_data.batch(64)
print('load val dataset shape', val_dataset.element_spec)

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
training_dataset = training_dataset.with_options(options)
validation_dataset = val_dataset.with_options(options)
# Have GPU prefetch next training batch while first one runs
training_dataset = training_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:

epochs: int = 100
callbacks = None
#setting validation data to None
history = mymodel.fit(x=training_dataset, validation_data=val_dataset, epochs=epochs,verbose=1, use_multiprocessing=False, workers=1, max_queue_size=5)
print(history)

# Extract the final training and validation loss
train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]


accuracy = mymodel.evaluate(x=training_dataset)
print('Accuracy:' ,accuracy)

mymodel.save(f'saved_models/fine_tuned_woscheduler_seed22222_lr0.0001_trainloss{train_loss:.5f}_valloss{val_loss:.5f}_addedhover160.h5')
